In [52]:
import PIL
import glob
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from IPython.display import Image
from PIL import Image as pili, ImageOps as piliops
from PIL import Image, ImageFont, ImageDraw, ImageFilter
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf

In [53]:
data = pd.read_csv('dialog.csv')
en_captions = data.iloc[:,0]
hi_captions = data.iloc[:,1]

In [54]:
font_size = 20
border = (30, 30)
font_color = (30, 30, 90)
TINT_COLOR = (0, 0, 0)  # Black
TRANSPARENCY = .25  # Degree of transparency, 0-100%
OPACITY = int(255 * TRANSPARENCY)

mypath = 'toons/comics'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
imlist = glob.glob("{}/test*.jpg".format(mypath))

cover = pili.open('toons/comics/test1.jpg')
width, height = cover.size

In [55]:
def hstacker(input_list):
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in input_list])[0][1]
    imgs_comb = np.hstack((np.asarray(i.resize(min_shape)) for i in input_list ) )
    imgs_comb = pili.fromarray(imgs_comb)
    return imgs_comb

In [56]:
def vstacker(input_list):
    min_shape = sorted([(np.sum(i.size), i.size ) for i in input_list])[0][1]
    imgs_comb = np.vstack((np.asarray( i.resize(min_shape) ) for i in input_list))
    imgs_comb = pili.fromarray( imgs_comb)
    return imgs_comb

In [57]:
def add_border_captions(img, caption, unicode=False):
    img = piliops.expand(pili.open(img), border=border,fill='white').convert('RGBA')
    overlay = pili.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    
    if unicode == True:
        font = pilf.truetype("c:/windows/fonts/nirmalas.ttf", font_size)
    else:
        font = pilf.truetype("arial.ttf", 30)
    
    w, h = font.getsize(caption)
    num_lines = len(caption.split('\n'))
    x, y = 0, img.height - (num_lines)*h
    
    draw.rectangle((x, y, x + img.width, y + (num_lines)*2*h), fill=TINT_COLOR+(OPACITY,))
    draw.text((x, y), caption, fill=font_color, font=font)
    
    img = pili.alpha_composite(img, overlay)
    img = img.convert("RGB")
    return img

In [60]:
from fpdf import FPDF
pdf_english = FPDF(unit = "pt", format = [width, height])
pdf_hindi = FPDF(unit = "pt", format = [width, height])

def arrange_images_to_pdfs(imlist, en_captions, hi_captions):
    while 0 < len(imlist):
        lines = 0
        num_cols = old_num_cols = 0
        while lines < 4 and 0 < len(imlist):
            lines += 1
            while num_cols == old_num_cols:
                num_cols = np.random.randint(3, 5)
            old_num_cols = num_cols
            window = imlist[:num_cols]
            print(window)
            
            en_window_captions, hi_window_captions = en_captions[:num_cols], hi_captions[:num_cols]
            en_window = [add_border_captions(i, encp) for i, encp in zip(window, en_window_captions) ]
            en_merged = hstacker(en_window)
            en_out_fname = 'toons/comics/temp_en{}.png'.format(lines)
            en_merged.save(en_out_fname)            
            pdf_english.add_page()
            pdf_english.image(en_out_fname, 0, 0, width, height)

            hi_window = [add_border_captions(i, hicp, unicode=True) for i, hicp in zip(window, hi_window_captions)]
            hi_merged = hstacker(hi_window)
            hi_out_fname = 'toons/comics/temp_hi{}.png'.format(lines)
            hi_merged.save(hi_out_fname)
            pdf_hindi.add_page()
            pdf_hindi.image(hi_out_fname, 0, 0, width, height)
            
            imlist = imlist[num_cols:]

arrange_images_to_pdfs(imlist, en_captions, hi_captions)
        
pdf_english.output("toons/comics/final_en.pdf", "F")
pdf_hindi.output("toons/comics/final_hi.pdf", "F")

print("Your comicbooks is ready!")

['toons/comics\\test1.jpg', 'toons/comics\\test2.jpg', 'toons/comics\\test3.jpg']


<ipython-input-55-4e6b642a4a8f>:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.hstack((np.asarray(i.resize(min_shape)) for i in input_list ) )


['toons/comics\\test4.jpg', 'toons/comics\\test5.jpg', 'toons/comics\\test6.jpg', 'toons/comics\\test7.jpg']
['toons/comics\\test8.jpg', 'toons/comics\\test9.jpg']
Your comicbooks is ready!
